## PGM Project

### Synthetic data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import warnings

plt.style.use("bmh")
sys.path.append("../src/")
warnings.filterwarnings("ignore")

from kernels import ExponentiatedQuadraticKernel
from MAGMA import MAGMA

In [2]:
def run(M=10, N=100, Ni=10, max_iterations=50):

    # 0. Base
    t0 = 0
    tN = 20
    common_T = np.linspace(t0, tN, N)
    a = np.random.uniform(-2, 2)
    b = np.random.uniform(0, 10)
    m0 = a * common_T + b
    m0_function = lambda t : a * t + b
    theta0 = np.array([np.random.uniform(1, np.exp(5)), np.random.uniform(1, np.exp(2))])
    K_theta0 = ExponentiatedQuadraticKernel.compute_all(theta0, common_T)
    mu0 = np.random.multivariate_normal(m0, K_theta0)

    # 1. Common HP
    Theta = np.array([np.random.uniform(1, np.exp(5)), np.random.uniform(1, np.exp(2))])
    Sigma = np.random.uniform(0, 1)

    # 1. 1. Common grid
    C_Theta = ExponentiatedQuadraticKernel.compute_all(Theta, common_T)
    Psi_Theta_Sigma = C_Theta + Sigma * np.identity(N)
    Y = np.zeros((M, N))
    for i in range(M):
        Yi = np.random.multivariate_normal(mu0, Psi_Theta_Sigma)
        Y[i] = Yi

    model_common_HP_common_grid = MAGMA(
        T=None,
        Y=Y,
        common_T=common_T,
        m0=np.zeros(len(m0)),
        m0_function=m0_function,
        theta0=np.array([np.random.uniform(0.98, 1.02), np.random.uniform(0.98, 1.02)]),
        Theta=np.array([np.random.uniform(0.98,1.02), np.random.uniform(0.98,1.02)]),
        Sigma=np.random.uniform(0.49,0.51),
        common_hp_flag=True,
        common_grid_flag=True,
        save_history_flag=False,
        scipy_optimize_display=False,
        kernel_k=ExponentiatedQuadraticKernel,
        kernel_c=ExponentiatedQuadraticKernel,
    )
    model_common_HP_common_grid.fit(max_iterations=max_iterations, eps=1e-2)

    # 1. 2. Uncommon grid
    T = np.zeros((M, Ni))
    Y = np.zeros((M, Ni))
    for i in range(M):
        Ti = np.sort(np.random.choice(common_T, size=Ni, replace=False))
        mask = np.isin(common_T, Ti)
        C_Theta = ExponentiatedQuadraticKernel.compute_all(Theta, Ti)
        Psi_Theta_Sigma = C_Theta + Sigma * np.identity(Ni)
        mu0_i = mu0[mask]
        Yi = np.random.multivariate_normal(mu0_i, Psi_Theta_Sigma)
        T[i] = Ti
        Y[i] = Yi

    model_common_HP_uncommon_grid = MAGMA(
        T=T,
        Y=Y,
        common_T=common_T,
        m0=np.zeros(len(m0)),
        m0_function=m0_function,
        theta0=np.array([np.random.uniform(0.98, 1.02), np.random.uniform(0.98, 1.02)]),
        Theta=np.array([np.random.uniform(0.98,1.02), np.random.uniform(0.98,1.02)]),
        Sigma=np.random.uniform(0.49,0.51),
        common_hp_flag=True,
        common_grid_flag=False,
        save_history_flag=False,
        scipy_optimize_display=False,
        kernel_k=ExponentiatedQuadraticKernel,
        kernel_c=ExponentiatedQuadraticKernel,
    )
    model_common_HP_uncommon_grid.fit(max_iterations=max_iterations, eps=1e-2)

    # 2. Different HP
    Theta = np.array([np.random.uniform(1, np.exp(5), size=M), np.random.uniform(1, np.exp(2), size=M)]).T
    Sigma = np.random.uniform(0, 1, size=M)

    # 2. 1. Common grid
    Y = np.zeros((M, N))
    for i in range(M):
        C_Theta = ExponentiatedQuadraticKernel.compute_all(Theta[i], common_T)
        Psi_Theta_Sigma = C_Theta + Sigma[i] * np.identity(N)
        Yi = np.random.multivariate_normal(mu0, Psi_Theta_Sigma)
        Y[i] = Yi

    model_different_HP_common_grid = MAGMA(
        T=None,
        Y=Y,
        common_T=common_T,
        m0=np.zeros(len(m0)),
        m0_function=m0_function,
        theta0=np.array([np.random.uniform(0.99,1.01), np.random.uniform(0.99,1.01)]),
        Theta=np.array([np.random.uniform(0.99,1.01, size=M), np.random.uniform(0.99,1.01, size=M)]).T,
        Sigma=np.random.uniform(0.49, 0.51, size=M),
        common_hp_flag=False,
        common_grid_flag=True,
        save_history_flag=False,
        scipy_optimize_display=False,
        kernel_k=ExponentiatedQuadraticKernel,
        kernel_c=ExponentiatedQuadraticKernel,
    )
    model_different_HP_common_grid.fit(max_iterations=max_iterations, eps=1e-2)

    # 2. 2. Uncommon grid
    T = np.zeros((M, Ni))
    Y = np.zeros((M, Ni))
    for i in range(M):
        Ti = np.sort(np.random.choice(common_T, size=Ni, replace=False))
        mask = np.isin(common_T, Ti)
        C_Theta = ExponentiatedQuadraticKernel.compute_all(Theta[i], Ti)
        Psi_Theta_Sigma = C_Theta + Sigma[i] * np.identity(Ni)
        mu0_i = mu0[mask]
        Yi = np.random.multivariate_normal(mu0_i, Psi_Theta_Sigma)
        T[i] = Ti
        Y[i] = Yi

    model_different_HP_uncommon_grid = MAGMA(
        T=T,
        Y=Y,
        common_T=common_T,
        m0=np.zeros(len(m0)),
        m0_function=m0_function,
        theta0=np.array([np.random.uniform(0.99,1.01), np.random.uniform(0.99,1.01)]),
        Theta=np.array([np.random.uniform(0.99,1.01, size=M), np.random.uniform(0.99,1.01, size=M)]).T,
        Sigma=np.random.uniform(0.49, 0.51, size=M),
        common_hp_flag=False,
        common_grid_flag=False,
        save_history_flag=False,
        scipy_optimize_display=False,
        kernel_k=ExponentiatedQuadraticKernel,
        kernel_c=ExponentiatedQuadraticKernel,
    )
    model_different_HP_uncommon_grid.fit(max_iterations=max_iterations, eps=1e-2)

    # 3. mu0 MSE
    mu0_mse = {
        "CHP_CG": ((mu0 - model_common_HP_common_grid.m0_estim)**2).sum()/(N * M),
        "CHP_UG": ((mu0 - model_common_HP_uncommon_grid.m0_estim)**2).sum()/(N * M),
        "DHP_CG": ((mu0 - model_different_HP_common_grid.m0_estim)**2).sum()/(N * M),
        "DHP_UG": ((mu0 - model_different_HP_uncommon_grid.m0_estim)**2).sum()/(N * M),
    }

    # 4. Predictions
    pred_mse = {
        "CHP_CG": 0,
        "CHP_UG": 0,
        "DHP_CG": 0,
        "DHP_UG": 0,
    }
    N_p = 10
    for i in range(N_p):
        Theta = np.array([np.random.uniform(1, np.exp(5)), np.random.uniform(1, np.exp(2))])
        Sigma = np.random.uniform(0, 1)
        T_p_obs = np.linspace((i) * 10, (i + 1) * 10, 2*Ni)
        T_obs = T_p_obs[:Ni]
        T_p = T_p_obs[Ni:]
        m_p_obs = m0_function(T_p_obs)
        m_P = m_p_obs[Ni:]
        Cov_p_obs = ExponentiatedQuadraticKernel.compute_all(Theta, T_p_obs) + Sigma * np.identity(2 * Ni)
        Y_p_obs = np.random.multivariate_normal(m_p_obs, Cov_p_obs)
        Y_obs = Y_p_obs[:Ni]
        Y_p = Y_p_obs[Ni:]

        m_P_pred, _ = model_common_HP_common_grid.predict(T_p, T_obs, Y_obs)
        mse = ((m_P_pred - m_P) ** 2).sum()/(N * M)
        pred_mse["CHP_CG"] += mse/N_p

        m_P_pred, _ = model_common_HP_uncommon_grid.predict(T_p, T_obs, Y_obs)
        mse = ((m_P_pred - m_P) ** 2).sum()/(N * M)
        pred_mse["CHP_UG"] += mse/N_p

        m_P_pred, _ = model_different_HP_common_grid.predict(T_p, T_obs, Y_obs)
        mse = ((m_P_pred - m_P) ** 2).sum()/(N * M)
        pred_mse["DHP_CG"] += mse/N_p

        m_P_pred, _ = model_different_HP_uncommon_grid.predict(T_p, T_obs, Y_obs)
        mse = ((m_P_pred - m_P) ** 2).sum()/(N * M)
        pred_mse["DHP_UG"] += mse/N_p

    return mu0_mse, pred_mse


In [3]:
n_runs = 10

all_mu0_mse = {
    "CHP_CG": [],
    "CHP_UG": [],
    "DHP_CG": [],
    "DHP_UG": [],
}

all_pred_mse = {
    "CHP_CG": [],
    "CHP_UG": [],
    "DHP_CG": [],
    "DHP_UG": [],
}

for _ in range(n_runs):
    mu0_mse, pred_mse = run()
    for config in all_mu0_mse.keys():
        all_mu0_mse[config].append(mu0_mse[config])
        all_pred_mse[config].append(pred_mse[config])

MAGMA Training: 100%|██████████| 50/50 [00:56<00:00,  1.13s/it]


In [4]:
for config in all_mu0_mse.keys():
    print(config)
    print(f"\tmu0: {np.mean(all_mu0_mse[config])} ({np.std(all_mu0_mse[config])})")
    print(f"\tpred: {np.mean(all_pred_mse[config])} ({np.std(all_pred_mse[config])})")

CHP_CG
	mu0: 65.86746224530017 (87.64268212144907)
	pred: 418.06262169091326 (691.0428816655749)
CHP_UG
	mu0: 67.78972139184606 (84.05343741445044)
	pred: 132.73408765329174 (137.91592495600014)
DHP_CG
	mu0: 38.55777899752914 (57.04239839287518)
	pred: 63.293389209738145 (34.63898529271923)
DHP_UG
	mu0: 149.2515827130063 (406.1355659929502)
	pred: 57.47477098114613 (32.89612503255965)
